In [55]:
DEGREE_N = 16
BIT_WIDTH = 64
Q = 1048193
t = 16
T = 256
L = 7

In [38]:
DEGREE_N = 64
BIT_WIDTH = 64
Q = 4294966657
t = 64
T = 16
L = 7
PATH = "HE_Verilog/tests/bins_demo/"

In [39]:
def poly_mod(A):
    D = []
    n = len(A) // 2
    for i in range(n):
        # print("{} - {} = {} -> ".format(hex(A[i]), hex(A[i+n]), hex(A[i] - A[i+n])), end='')
        D.append((A[i] - A[i + n]) % Q)
        # print(hex(D[i]))
        
    return [(d+Q) if d < 0 else d for d in D]

In [40]:
def load_inputs(filename):
    c = []
    with open(filename, "rb") as f:
        for i in range(DEGREE_N):
            c.append(int.from_bytes(f.read(BIT_WIDTH // 8), "little"))
    return c

In [41]:
def load_relin_keys(filename):
    with open(filename, "rb") as f:
        C = []
        for i in range(L+1):
            C.append([])
            for j in range(2):
                C[i].append([])
                for k in range(DEGREE_N):
                    # C[i][j][k] = int.from_bytes(f.read(BIT_WIDTH // 8), "little")
                    C[i][j].append(int.from_bytes(f.read(BIT_WIDTH // 8), "little"))
        return C

In [42]:
def show(ctxt):
    print([hex(x) for x in ctxt])

In [43]:
def poly_mul(A, B):
    C = [0] * (2 * len(A))
    for indexA, elemA in enumerate(A):
        for indexB, elemB in enumerate(B):
            C[indexA + indexB] = (C[indexA + indexB] + elemA * elemB) # % Q
    return C

In [44]:
def recon(A):
    A = [((a*t)//Q)%Q for a in A]
    return [(a+Q) if a < 0 else a for a in A]

In [45]:
def verify(A, B):
    for a, b in zip(A, B):
        try:
            assert (abs(a-b) < 5)
        except AssertionError:
            print(a, b)

In [46]:
def base_t(A):
    C = [[0]*DEGREE_N]*(L+1)
    for i in range(L+1):
        for j in range(DEGREE_N):
            C[i][j] = (A[j] // (T**i)) % T
        C[i] = [(c+T) if c < 0 else c for c in C[i]]
    return C

In [57]:
c00 = load_inputs(PATH + "ct10_fresh.bin")
c01 = load_inputs(PATH + "ct11_fresh.bin")
c10 = load_inputs(PATH + "ct20_fresh.bin")
c11 = load_inputs(PATH + "ct21_fresh.bin")
rlks = load_relin_keys(PATH + "relinKey.bin")

In [58]:
c0_expected = load_inputs("HE_Verilog/tests/bins_20_16/ct_afterMul_0.bin")
c1_expected = load_inputs("HE_Verilog/tests/bins_20_16/ct_afterMul_1.bin")
c2_expected = load_inputs("HE_Verilog/tests/bins_20_16/ct_afterMul_2.bin")

In [59]:
c0 = poly_mod(recon(poly_mul(c00, c10)))
c1 = poly_mod(recon(poly_mul(c00, c11) + poly_mul(c01, c10)))
c2 = poly_mod(recon(poly_mul(c01, c11)))
show(c2)

['0x88383', '0x87056', '0x3e483', '0x74ed4', '0xa35db', '0x13e2a', '0x98b44', '0x4d0a8', '0x48a0f', '0x6a3df', '0x94d6b', '0x6eb57', '0x12bb8', '0x744c7', '0x59edb', '0xb0367']


In [60]:
c0 = load_inputs("HE_Verilog/tests/bins_20_16/c0_wrapper.bin")
c1 = load_inputs("HE_Verilog/tests/bins_20_16/c1_wrapper.bin")
c2 = load_inputs("HE_Verilog/tests/bins_20_16/c2_wrapper.bin")
show(c1)
show(c1_expected)

['0x29e6f', '0x76ebc', '0x1a902', '0x4e5b1', '0x70ef4', '0x18a25', '0x660ca', '0xfff54d96', '0x577db', '0x66b99', '0x14a0', '0x2aa2f', '0x11d05e', '0x347c5', '0xfe25d', '0x179711']
['0x29e6f', '0x76ebc', '0x1a902', '0x4e5b2', '0x70ef4', '0x18a26', '0x660ca', '0x54c17', '0x577db', '0x66b9a', '0x149f', '0x2aa2e', '0x1d1dd', '0x347c4', '0xfe25d', '0x79891']


In [14]:
# verify(c0, c0_expected)
verify(c1, c1_expected)
# verify(c2, c2_expected)

4294266262 347159
1167454 119261
1546001 497809


In [61]:
show(c0_expected)
show(c1_expected)
show(c2_expected)

['0x4b1aa', '0x7478c', '0xb9cb5', '0x43de5', '0x32df7', '0xf0907', '0x84cb2', '0xd6dd3', '0x85234', '0x880a2', '0x9f779', '0xa5298', '0xfd7ab', '0xd7bc9', '0x8faa4', '0x1c078']
['0x29e6f', '0x76ebc', '0x1a902', '0x4e5b2', '0x70ef4', '0x18a26', '0x660ca', '0x54c17', '0x577db', '0x66b9a', '0x149f', '0x2aa2e', '0x1d1dd', '0x347c4', '0xfe25d', '0x79891']
['0xcda44', '0xf94', '0xd8923', '0xf1e61', '0xb4f80', '0x9ffc9', '0x790dc', '0x68286', '0x545cc', '0xb2830', '0xa53c5', '0xc6c9a', '0xddf52', '0xb2c4d', '0x8f852', '0xaa9bc']


In [62]:
c2_base_t = base_t(c2_expected)
for i in range(L+1):
    show(c2_base_t[i])

['0x44', '0x94', '0x23', '0x61', '0x80', '0xc9', '0xdc', '0x86', '0xcc', '0x30', '0xc5', '0x9a', '0x52', '0x4d', '0x52', '0xbc']
['0xda', '0xf', '0x89', '0x1e', '0x4f', '0xff', '0x90', '0x82', '0x45', '0x28', '0x53', '0x6c', '0xdf', '0x2c', '0xf8', '0xa9']
['0xc', '0x0', '0xd', '0xf', '0xb', '0x9', '0x7', '0x6', '0x5', '0xb', '0xa', '0xc', '0xd', '0xb', '0x8', '0xa']
['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']


In [63]:
def relin():
    C = [0]*(L+1)
    D = [0]*(L+1)
    for i in range(L+1):
        C[i] = poly_mul(c2_base_t[i], rlks[i][0])
        D[i] = poly_mul(c2_base_t[i], rlks[i][1])
        
    C = [poly_mod(c) for c in C]
    D = [poly_mod(d) for d in D]
    
    
    c0_out = [0 for c in c0_expected]
    c1_out = [0 for c in c1_expected]
    for i in range(L+1):
        for j in range(DEGREE_N):
            c0_out[j] += C[i][j]
            c1_out[j] += D[i][j]
    
    
    
    c0_out = [(c + r) % Q for c, r in zip(c0_expected, c0_out)]
    c1_out = [(c + r) % Q for c, r in zip(c1_expected, c1_out)]

    
    return c0_out, c1_out

In [64]:
c0_out, c1_out = relin()
show(c0_out)
show(c1_out)

['0xd383c', '0xdff2', '0x4b83c', '0xd7ff', '0xe1d27', '0xc91ce', '0xa6b75', '0x75192', '0xaabec', '0x9743e', '0xca940', '0xe5c8a', '0xba0bc', '0xf9809', '0xfd36e', '0x94dbe']
['0x7fee', '0x89982', '0xb76c1', '0x5cf86', '0x8da8b', '0x7907e', '0xbefb8', '0xd7dcd', '0x35d30', '0x9ed37', '0xdfc14', '0x65ad9', '0xc167f', '0xb22da', '0x5761d', '0x18ab8']


In [65]:
c0_final = load_inputs("HE_Verilog/tests/bins_20_16/ct_afterRelin_0.bin")
ctR0 = load_inputs("HE_Verilog/tests/ctR0.bin")
ctR1 = load_inputs("HE_Verilog/tests/ctR1.bin")
c1_final = load_inputs("HE_Verilog/tests/bins_20_16/ct_afterRelin_1.bin")

show(ctR0)
show(ctR1)

['0x3a92a', '0x3587c', '0xbb675', '0x6d97e', '0xa16f3', '0x487d3', '0x1e7d7', '0x8e6fe', '0x5e647', '0x8bad7', '0x9c3c8', '0xb19a8', '0xf17e4', '0xcb512', '0x6ea37', '0xf9aa9']
['0xb8ac1', '0xb7a59', '0x97b86', '0xcf36', '0x4b476', '0x4d1d', '0x7ca85', '0xf46ae', '0x394f5', '0xb9d2b', '0xe7fd0', '0x174c9', '0xc19ab', '0xa741a', '0x62185', '0x3445a']


In [66]:
ctR0 = load_inputs("HE_Verilog/tests/ctR0.bin")
show(ctR0)

['0x3a92a', '0x3587c', '0xbb675', '0x6d97e', '0xa16f3', '0x487d3', '0x1e7d7', '0x8e6fe', '0x5e647', '0x8bad7', '0x9c3c8', '0xb19a8', '0xf17e4', '0xcb512', '0x6ea37', '0xf9aa9']
